In [9]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Float, text
import pandas as pd

files = [
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_DATASET_cleaned.csv", "Dataset"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_EMSC_cleaned.csv", "EMSC"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_GEOFON_cleaned.csv", "GEOFON"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_USGS_cleaned.csv", "USGS")
]

dfs = []

for file_path, source_name in files:
    df = pd.read_csv(file_path)

    if 'place' in df.columns and 'region' in df.columns:
        print(f"Both 'place' and 'region' found in {source_name}, keeping 'region' only.")
        df = df.drop(columns=['place'])

    if 'latitude' in df.columns and 'longitude' in df.columns:
        df['coordination'] = df.apply(lambda x: f"[{x.latitude}, {x.longitude}]", axis=1)

    rename_map = {}
    if 'mag' in df.columns:
        rename_map['mag'] = 'magnitude'
    if 'place' in df.columns and 'region' not in df.columns:
        rename_map['place'] = 'region'

    df = df.rename(columns=rename_map)

    if 'source' in df.columns:
        df = df.drop(columns=['source'])

    df['source'] = source_name

    columns_needed = ['time', 'coordination', 'depth', 'magnitude', 'region', 'source']
    df = df[[c for c in columns_needed if c in df.columns]]

    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.date
    if 'depth' in df.columns:
        df['depth'] = pd.to_numeric(df['depth'], errors='coerce')
    if 'magnitude' in df.columns:
        df['magnitude'] = pd.to_numeric(df['magnitude'], errors='coerce')

    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

engine = create_engine('sqlite:///mydb.db')
connection = engine.connect()
metadata = MetaData()

earthquakes = Table(
    'Earthquakes',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('time', Date),
    Column('coordination', String),
    Column('depth', Float),
    Column('magnitude', Float),
    Column('region', String),
    Column('source', String)
)

connection.execute(text("DROP TABLE IF EXISTS Earthquakes;"))
metadata.create_all(engine)

df_all.to_sql(
    name='Earthquakes',
    con=engine,
    if_exists='append',
    index=False,
    chunksize=1000
)

count = connection.execute(text("SELECT COUNT(*) FROM Earthquakes;")).scalar()
print(f"Number of rows inserted: {count}")

result = connection.execute(
    text("SELECT id, time, coordination, depth, magnitude, region, source FROM Earthquakes LIMIT 5;")
).fetchall()

for row in result:
    print(row)
    

Both 'place' and 'region' found in Dataset, keeping 'region' only.
Both 'place' and 'region' found in EMSC, keeping 'region' only.
Both 'place' and 'region' found in GEOFON, keeping 'region' only.
Both 'place' and 'region' found in USGS, keeping 'region' only.
Number of rows inserted: 217
(1, '2025-09-15', '[38.322, 142.369]', 35.0, 5.1, 'Honshu', 'Dataset')
(2, '2025-09-16', '[36.2048, 138.2529]', 0.0, 4.866666666666666, 'Nagano', 'Dataset')
(3, '2025-09-18', '[33.5904, 130.4017]', 12.0, 4.866666666666666, 'Fukuoka', 'Dataset')
(4, '2025-09-19', '[35.6895, 139.6917]', 0.0, 5.5, 'Tokyo', 'Dataset')
(5, '2025-09-20', '[37.765, 140.467]', 0.0, 4.6, 'Fukushima', 'Dataset')


In [8]:
connection.execute(text("DROP TABLE IF EXISTS Earthquakes;"))


In [4]:
df.head()

,time,coordination,depth,magnitude,region,source
0,2025-10-10,"[41.7258, 142.679]",58.696,4.3,Urakawa,USGS
1,2025-10-07,"[29.2431, 129.3903]",10.000,5.3,Tatsugō,USGS
2,2025-10-07,"[37.6701, 144.1112]",10.000,4.6,Onagawa Chō,USGS
3,2025-10-07,"[37.8333, 144.1149]",10.000,4.7,Onagawa Chō,USGS
4,2025-10-07,"[37.5675, 141.3033]",49.324,5.1,Namie,USGS


In [5]:
df.tail()

,time,coordination,depth,magnitude,region,source
34,2025-09-15,"[37.6698, 138.8388]",185.842,4.3,Yoshida-kasugachō,USGS
35,2025-09-15,"[38.5338, 141.9747]",62.107,4.4,Onagawa Chō,USGS
36,2025-09-13,"[42.0242, 137.5574]",286.491,4.1,Kaminokuni,USGS
37,2025-09-13,"[34.1815, 137.2194]",333.697,4.1,Shima,USGS
38,2025-09-13,"[32.867, 141.7432]",10.000,4.9,Katsuura,USGS


In [15]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Float, text
import pandas as pd

# 📂 مسیر فایل‌ها و نام منبع‌ها
files = [
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_DATASET_cleaned.csv", "Dataset"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_EMSC_cleaned.csv", "EMSC"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_GEOFON_cleaned.csv", "GEOFON"),
    ("/Users/Vengeance/Documents/GitHub/earthquake-analysis-japan/JAPAN_USGS_cleaned.csv", "USGS")
]

dfs = []

# 🧩 خواندن و آماده‌سازی هر فایل
for file_path, source_name in files:
    df = pd.read_csv(file_path)

    # اگه place و region با هم بودن، یکی رو حذف کن
    if 'place' in df.columns and 'region' in df.columns:
        df = df.drop(columns=['place'])

    # ساخت ستون مختصات
    if 'latitude' in df.columns and 'longitude' in df.columns:
        df['coordination'] = df.apply(lambda x: f"[{x.latitude}, {x.longitude}]", axis=1)

    # تغییر نام ستون‌ها
    rename_map = {}
    if 'mag' in df.columns:
        rename_map['mag'] = 'magnitude'
    if 'place' in df.columns and 'region' not in df.columns:
        rename_map['place'] = 'region'
    df = df.rename(columns=rename_map)

    # افزودن منبع داده درست برای هر فایل
    df['source'] = source_name

    # فقط ستون‌های مورد نیاز
    columns_needed = ['time', 'coordination', 'depth', 'magnitude', 'region', 'source']
    df = df[[c for c in columns_needed if c in df.columns]]

    # تبدیل نوع داده‌ها
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.date
    if 'depth' in df.columns:
        df['depth'] = pd.to_numeric(df['depth'], errors='coerce')
    if 'magnitude' in df.columns:
        df['magnitude'] = pd.to_numeric(df['magnitude'], errors='coerce')

    dfs.append(df)

# 🔗 ادغام همه‌ی فایل‌ها
df_all = pd.concat(dfs, ignore_index=True)

# 🧹 حذف ردیف‌های ناقص یا تکراری
df_all = df_all.dropna(subset=['time', 'magnitude', 'region'])
df_all = df_all.drop_duplicates()

# 🧱 اتصال به دیتابیس و ساخت جدول
engine = create_engine('sqlite:///mydb.db')
connection = engine.connect()
metadata = MetaData()

earthquakes = Table(
    'Earthquakes',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('time', Date),
    Column('coordination', String),
    Column('depth', Float),
    Column('magnitude', Float),
    Column('region', String),
    Column('source', String)
)

# حذف جدول قبلی و ساخت مجدد
connection.execute(text("DROP TABLE IF EXISTS Earthquakes;"))
metadata.create_all(engine)

# 💾 ذخیره داده‌ها در جدول
df_all.to_sql(
    name='Earthquakes',
    con=engine,
    if_exists='append',
    index=False,
    chunksize=1000
)

# 📊 تعداد رکوردها
count = connection.execute(text("SELECT COUNT(*) FROM Earthquakes;")).scalar()
print(f"✅ Number of rows inserted: {count}")

# 🧾 نمایش چند ردیف اول
result = connection.execute(
    text("SELECT id, time, coordination, depth, magnitude, region, source FROM Earthquakes LIMIT 5;")
).fetchall()

for row in result:
    print(row)

✅ Number of rows inserted: 213
(1, '2025-09-15', '[38.322, 142.369]', 35.0, 5.1, 'Honshu', 'Dataset')
(2, '2025-09-16', '[36.2048, 138.2529]', 0.0, 4.866666666666666, 'Nagano', 'Dataset')
(3, '2025-09-18', '[33.5904, 130.4017]', 12.0, 4.866666666666666, 'Fukuoka', 'Dataset')
(4, '2025-09-19', '[35.6895, 139.6917]', 0.0, 5.5, 'Tokyo', 'Dataset')
(5, '2025-09-20', '[37.765, 140.467]', 0.0, 4.6, 'Fukushima', 'Dataset')


In [23]:
df_all.head(213)

,time,coordination,depth,magnitude,region,source
0,2025-09-15,"[38.322, 142.369]",35.000,5.100000,Honshu,Dataset
1,2025-09-16,"[36.2048, 138.2529]",0.000,4.866667,Nagano,Dataset
2,2025-09-18,"[33.5904, 130.4017]",12.000,4.866667,Fukuoka,Dataset
3,2025-09-19,"[35.6895, 139.6917]",0.000,5.500000,Tokyo,Dataset
4,2025-09-20,"[37.765, 140.467]",0.000,4.600000,Fukushima,Dataset
...,...,...,...,...,...,...
212,2025-09-15,"[37.6698, 138.8388]",185.842,4.300000,Yoshida-kasugachō,USGS
213,2025-09-15,"[38.5338, 141.9747]",62.107,4.400000,Onagawa Chō,USGS
214,2025-09-13,"[42.0242, 137.5574]",286.491,4.100000,Kaminokuni,USGS
215,2025-09-13,"[34.1815, 137.2194]",333.697,4.100000,Shima,USGS


In [18]:
df.tail()

,time,coordination,depth,magnitude,region,source
34,2025-09-15,"[37.6698, 138.8388]",185.842,4.3,Yoshida-kasugachō,USGS
35,2025-09-15,"[38.5338, 141.9747]",62.107,4.4,Onagawa Chō,USGS
36,2025-09-13,"[42.0242, 137.5574]",286.491,4.1,Kaminokuni,USGS
37,2025-09-13,"[34.1815, 137.2194]",333.697,4.1,Shima,USGS
38,2025-09-13,"[32.867, 141.7432]",10.000,4.9,Katsuura,USGS


In [26]:
result = connection.execute(text("SELECT * FROM Earthquakes")).fetchall()
for row in result:
    print(row)

(1, '2025-09-15', '[38.322, 142.369]', 35.0, 5.1, 'Honshu', 'Dataset')
(2, '2025-09-16', '[36.2048, 138.2529]', 0.0, 4.866666666666666, 'Nagano', 'Dataset')
(3, '2025-09-18', '[33.5904, 130.4017]', 12.0, 4.866666666666666, 'Fukuoka', 'Dataset')
(4, '2025-09-19', '[35.6895, 139.6917]', 0.0, 5.5, 'Tokyo', 'Dataset')
(5, '2025-09-20', '[37.765, 140.467]', 0.0, 4.6, 'Fukushima', 'Dataset')
(6, '2025-09-21', '[34.6937, 135.5023]', 45.1, 5.2, 'Osaka', 'Dataset')
(7, '2025-09-23', '[39.702, 141.152]', 50.5, 4.866666666666666, 'Sendai', 'Dataset')
(8, '2025-09-24', '[35.4437, 139.638]', 30.0, 5.0, 'Yokohama', 'Dataset')
(9, '2025-09-26', '[26.2124, 127.6809]', 15.0, 4.7, 'Ryukyu Islands', 'Dataset')
(10, '2025-09-28', '[40.822, 140.747]', 60.1, 5.0, 'Aomori', 'Dataset')
(11, '2025-09-30', '[34.3852, 132.4553]', 80.0, 4.866666666666666, 'Hiroshima', 'Dataset')
(12, '2025-10-01', '[35.0116, 135.7681]', 10.0, 4.4, 'Kyoto', 'Dataset')
(13, '2025-10-02', '[32.7503, 129.8779]', 25.3, 4.9, 'Nagasaki'